<b>LinkedIn рестарт (М1)</b>

<b>Проект: Анализ вакансии для джуниоров аналитиков данных</b><br>
Цель: Визуализировать информацию о рынке вакансий для аналитиков в Европе<br>
Источник данных: Датасет Аналитик + bi аналитик (джун).<br>

<b>Описание данных</b><br>
job_title - название вакансии<br>
area - полное название места работы<br>
city - город места работы<br>
country - страна места работы<br>
workplace - тип занятости<br>
company_name - название компании-работодателя<br>
ndustry - сфера деятельности компании-работодателя<br>
company_size - размер компании-работодателя<br>
job_description - описание вакансии<br>
posted_date - дата публикации вакансии<br>
applicants - количество откликов на вакансию<br>

<b>ТЗ</b><br>
Подготовка данных<br>
1. Проверить данные на дубликаты.<br>
2. Обработать пропуски.<br>
3. Удалить не релевантные заданию вакансии.<br>
4. Привести столбец с датой публикации к виду, пригодному для обработки.<br>
5. Выделить hard skills из описания вакансии — их проще искать в разноязычных
вакансиях, чем soft skills.<br>
6. Удалить ненужные атрибуты (признаки).<br>

# Подготовка данных

In [92]:
# Необходимые библиотеки для исследования 
import pandas as pd
import requests
import numpy as np
from datetime import datetime, timedelta
from io import StringIO
from urllib.parse import urlencode
from typing import List
import warnings
warnings.filterwarnings('ignore')

In [93]:
url='https://drive.google.com/file/d/XXX/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
try:
    df = pd.read_csv(r'C:\Users\1\DATA\masterskaya_LinkedIn_2023_05.csv')
except:
    df = pd.read_csv(url, index_col=0)

df.head(5)

,job_title,area,city,country,workplace,company_name,industry,company_size,job_description,posted_date,applicants
0,Data Analyst,"['Basel', 'Basel', 'Switzerland']",Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,Staffing & Recruiting,11-50,About the job\n \n\n \n ...,1 week ago,47.0
1,Data Analyst - Logistics,"['Coventry', 'England', 'United Kingdom']",Coventry,United Kingdom,On-site,Resolute Recruitment,NaN,NaN,About the job,1 week ago,NaN
2,Data Analyst - Logistics,"['Coventry', 'England', 'United Kingdom']",Coventry,United Kingdom,On-site,Resolute Recruitment,NaN,NaN,About the job\n \n\n \nData Analyst - L...,1 week ago,NaN
3,Data Analyst (Space & Planning),"['South Molton', 'England', 'United Kingdom']",South Molton,United Kingdom,On-site,Mole Valley Farmers,NaN,NaN,About the job\n \n\n \nSalary: To be di...,1 week ago,NaN
4,Data Analyst,"['Lugano', 'Ticino', 'Switzerland']",Lugano,Switzerland,On-site,FORFIRM,NaN,NaN,About the job\n \n\n \nFORFIRM is provi...,2 weeks ago,NaN


In [94]:
df.info()
df.posted_date.unique ()

<class 'pandas.core.frame.DataFrame'>
Index: 998 entries, 0 to 997
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   job_title        998 non-null    object 
 1   area             998 non-null    object 
 2   city             998 non-null    object 
 3   country          998 non-null    object 
 4   workplace        930 non-null    object 
 5   company_name     996 non-null    object 
 6   industry         964 non-null    object 
 7   company_size     963 non-null    object 
 8   job_description  998 non-null    object 
 9   posted_date      998 non-null    object 
 10  applicants       838 non-null    float64
dtypes: float64(1), object(10)
memory usage: 93.6+ KB


array(['1 week ago', '2 weeks ago', '6 days ago', '3 weeks ago',
       '2 days ago', '1 day ago', '4 days ago', '4 weeks ago',
       '3 days ago', '5 days ago', '12 minutes ago', '29 minutes ago',
       '5 hours ago', '8 hours ago', '6 hours ago', '9 hours ago',
       '11 hours ago', '12 hours ago', '7 hours ago', '10 hours ago'],
      dtype=object)

На данном этапе мы загрузили необходимые библиотеки и файл данных, далее последует анализ данных.

### Проверим дубликаты

In [95]:
# проверим наличие полных дубликатов
print('Найдено',df.duplicated().sum(),'полных дубликатов')
df = df.drop_duplicates()

Найдено 111 полных дубликатов


In [96]:
# проверим неявные дубликаты 
dpl=df.duplicated(subset=['job_title', 'company_name','city']).sum()
print('При поиске по связке название вакансии и организация, найдено',dpl, 'неявных дубликатов')

При поиске по связке название вакансии и организация, найдено 8 неявных дубликатов


In [97]:
df = df.drop_duplicates(subset=['job_title', 'company_name'])

На этом этапе было выявлено 111 полных и 96 не полных дубликатов.

### Обработаем пропуски.

In [98]:
df.isna().sum()

job_title            0
area                 0
city                 0
country              0
workplace           59
company_name         2
industry            30
company_size        31
job_description      0
posted_date          0
applicants         125
dtype: int64

In [99]:
# Удалим компании без названия
df = df.dropna(subset=['company_name'])

In [100]:
# заполним нулями количесвто претендентов
df['applicants'] = df['applicants'].fillna(0)

In [101]:
# поставим заглушку пропусков "not_set" для workplace, company_size, industry
df['workplace'] = df['workplace'].fillna("not_set")
df['company_size'] = df['company_size'].fillna("not_set")
df['industry'] = df['industry'].fillna("not_set")
#проверим
df.isna().sum()

job_title          0
area               0
city               0
country            0
workplace          0
company_name       0
industry           0
company_size       0
job_description    0
posted_date        0
applicants         0
dtype: int64

In [102]:
print("после фильтрации осталось",df.shape[0]," записей")

после фильтрации осталось 789  записей


На этом этапе мы обработали пропуски, для столбцов workplace, company_size, industry их заменили на заглушку "not_set"
В итоге осталось 789 записей без пропусков.

### Удалим не релевантные заданию вакансии.

Задача непростая, если просто выбрать всех джунов, получим всего 59 записей, бдуем идти от обратного, если нет требования middle
senior или teamlead, считаем вакансию пригодной для джуна, для этого добавим новый столбец и будем задавать его 0 или 1

In [103]:
df.job_title.unique ()

array(['Data Analyst', 'Data Analyst - Logistics',
       'Data Analyst (Space & Planning)', 'Data Analyst - Hybrid Working',
       'Commercial Data Analyst', 'Asset Data Analyst',
       'Data Analyst - Hybrid', 'Data Analyst (FT)', 'Data Analyst H/F',
       'Data Analyst II', 'Data Analyst (9 Months FTC)',
       'Business Intelligence Analyst', 'Data Analyst (F/H)',
       'BI Analyst', 'Data Analyst Associate', 'Product Data Analyst',
       'Graduate Data Analyst', 'Digital Data Analyst',
       'Business Intelligence Analyst Junior', 'Data analyst H/F',
       'Data Analyst (m/f/d)', 'Analytics Consultant', 'Data Analyst F/H',
       'Consultor BI', 'Statistical Data Analyst',
       'Digital Data Analyst (H/F)',
       'Data Analyst - €60,- per hour - Amsterdam based',
       'Data & Analytics Senior Analyst',
       'JUNIOR DATA ANALYST – Aerospace', 'Data Analyst - Operations',
       'Data Analyst (H/F)', 'BUSINESS INTELLIGENCE ANALYST',
       'Business Intelligence Analys

Создадим два массива (numpy.ndarray) r1 и r2, уникальных job_title, чтобы проверить какие job_title удалятся из датасета в итоге:

In [104]:
r1 = df.job_title.unique()

In [105]:
def check_word(row):
    if (words[0] in row['job_title'].lower()):
        if (words[2] in row['job_title'].lower()): 
            return 1
        else: 
            return 0
    elif words[1] in row['job_title'].lower():
        if (words[2] in row['job_title'].lower()): 
            return 1
        else: 
            return 0
    elif words[3] in row['job_title'].lower():
        if (words[2] in row['job_title'].lower()): 
            return 1
        else: 
            return 0
    elif words[4] in row['job_title'].lower():
        if (words[2] in row['job_title'].lower()): 
            return 1
        else: 
            return 0
    elif words[2] in row['job_title'].lower():
        return 1
  
    
words= ['middle','team lead','junior','senior','medior']
df['for_junior'] = df.apply(lambda row: check_word(row), axis=1)
df = df[~(df['for_junior'] == 0)]
print("после фильтрации осталось",df.shape[0]," записей")

после фильтрации осталось 762  записей


In [106]:
#Создаем список r2 - оставшихся job_title:
r2 = df.job_title.unique()

Выводим job_title, которые удалили из датасета. 

In [107]:
# Выводим элементы которые есть в r1, но нет в r2:
np.setdiff1d(r1,r2)

array(['(Senior) Consultant Transaction Analytics (f/m/d)',
       'BUSINESS DATA ANALYSTE Senior H/F', 'Business Analyst Middle',
       'Clinical Data Insights Analyst (Medior/Senior)',
       'Consultant Senior Microsoft BI - Power BI Expert H/F',
       'Data & Analytics Senior Analyst',
       'Data Analysis and Reporting Team Lead', 'Ingénieur Senior BI F/H',
       'Medior Data analyst', 'SENIOR BUSINESS REPORTING SPECIALIST',
       'Senior BI Consultant', 'Senior BI Data Engineer',
       'Senior BI Developer (SQL Server) – (Remote)',
       'Senior Business Intelligence Consultant', 'Senior Data Analyst',
       'Senior Data Analyst (Bangkok Based, Relocation Provided)',
       'Senior Data Analyst (f/m/d) - onsite or remote / home office',
       'Senior Data Scientist', 'Senior Datenanalyst (m/w/d)',
       'Senior Digital Analyst (*all genders)',
       'Senior Expert Innovation Analytics Performance',
       'Senior Financial Analyst (M/F)', 'Senior Product Analyst',
    

На данном этаме мы удалили вакансии, не подходящие для Junior.

### Приведем столбец с датой публикации к виду, пригодному для обработки.

In [108]:
df.posted_date.unique ()

array(['1 week ago', '2 weeks ago', '6 days ago', '3 weeks ago',
       '2 days ago', '1 day ago', '4 days ago', '4 weeks ago',
       '3 days ago', '5 days ago', '12 minutes ago', '29 minutes ago',
       '5 hours ago', '8 hours ago', '6 hours ago', '9 hours ago',
       '11 hours ago', '12 hours ago', '7 hours ago', '10 hours ago'],
      dtype=object)

В таком виде нам неудобно работать со столбцом, тут есть и минуты и недели и дни. Приведем все значения в timedelta.

In [109]:
def search(x):
    if isinstance(x, int):
        time=x
        return time
    elif 'week' in x:
        time = timedelta(int(x[0])*7)
        return time
    elif 'day' in x:
        time = timedelta(int(x[0]))
        return time
    elif 'hours' in x:
        time = timedelta(hours=int(x[0:2]))
        return time
    elif 'minutes' in x:
        time = timedelta(minutes=int(x[0:2]))
        return time
    return x

df['posted_date'] = df['posted_date'].apply(search)

df.posted_date.unique ()

<TimedeltaArray>
[ '7 days 00:00:00', '14 days 00:00:00',  '6 days 00:00:00',
 '21 days 00:00:00',  '2 days 00:00:00',  '1 days 00:00:00',
  '4 days 00:00:00', '28 days 00:00:00',  '3 days 00:00:00',
  '5 days 00:00:00',  '0 days 00:12:00',  '0 days 00:29:00',
  '0 days 05:00:00',  '0 days 08:00:00',  '0 days 06:00:00',
  '0 days 09:00:00',  '0 days 11:00:00',  '0 days 12:00:00',
  '0 days 07:00:00',  '0 days 10:00:00']
Length: 20, dtype: timedelta64[ns]

In [110]:
df['date']=datetime.now()
for i in range(len(df)):
    df['date'].iloc[i]= datetime.today() - df.posted_date.iloc[i]   
df.head(5)

,job_title,area,city,country,workplace,company_name,industry,company_size,job_description,posted_date,applicants,for_junior,date
0,Data Analyst,"['Basel', 'Basel', 'Switzerland']",Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,Staffing & Recruiting,11-50,About the job\n \n\n \n ...,7 days,47.0,NaN,2024-02-12 16:06:39.175584
1,Data Analyst - Logistics,"['Coventry', 'England', 'United Kingdom']",Coventry,United Kingdom,On-site,Resolute Recruitment,not_set,not_set,About the job,7 days,0.0,NaN,2024-02-12 16:06:39.176582
3,Data Analyst (Space & Planning),"['South Molton', 'England', 'United Kingdom']",South Molton,United Kingdom,On-site,Mole Valley Farmers,not_set,not_set,About the job\n \n\n \nSalary: To be di...,7 days,0.0,NaN,2024-02-12 16:06:39.176582
4,Data Analyst,"['Lugano', 'Ticino', 'Switzerland']",Lugano,Switzerland,On-site,FORFIRM,not_set,not_set,About the job\n \n\n \nFORFIRM is provi...,14 days,0.0,NaN,2024-02-05 16:06:39.177600
5,Data Analyst - Logistics,"['Southampton', 'England', 'United Kingdom']",Southampton,United Kingdom,On-site,"Butler, Bridge & May",not_set,not_set,About the job\n \n\n \nLocation: Southa...,6 days,0.0,NaN,2024-02-13 16:06:39.177600


На этом этапе мы перевели время в timedelta, смещение от даты выгрузки датасета и добавили отдельный столбец разницы текущего времени и времени публикации. Он нам пригодится для визуализации данных.

### Выделим hard skills из описания вакансии

In [111]:
#Указываем список некоторых навыков:
word = ['sql', 'python', 'data visualization', 'tableau', 'power bi']

# Создаем функцию для сбора найденных в описании вакансии навыков в список:
def search_skills(rows):
    #Создаем пустой список:
    skills_list = []
    # Для каждого навыка:
    for i in word:
        #Проверяем есть ли навык в строке:
        if i in rows:
            # Если есть добавляем его в список:
            skills_list.append(i)
    # Возвращаем список:
    return skills_list

In [112]:
# Вызываем функцию для нового столбца - skills:
df['skills'] = df['job_description'].apply(lambda x: search_skills(x))

Далее если список скилов состоит из 2-х и более навыков, разобъем список на строки,
в итоге получится столько строк одной вакансии, сколько навыков она содержит:

In [113]:
df = df.explode('skills', ignore_index=True) 
# ignore_index=True - указываем для того чтобы каждая строка получила уникальный индекс
df.head(5)

,job_title,area,city,country,workplace,company_name,industry,company_size,job_description,posted_date,applicants,for_junior,date,skills
0,Data Analyst,"['Basel', 'Basel', 'Switzerland']",Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,Staffing & Recruiting,11-50,About the job\n \n\n \n ...,7 days,47.0,NaN,2024-02-12 16:06:39.175584,NaN
1,Data Analyst - Logistics,"['Coventry', 'England', 'United Kingdom']",Coventry,United Kingdom,On-site,Resolute Recruitment,not_set,not_set,About the job,7 days,0.0,NaN,2024-02-12 16:06:39.176582,NaN
2,Data Analyst (Space & Planning),"['South Molton', 'England', 'United Kingdom']",South Molton,United Kingdom,On-site,Mole Valley Farmers,not_set,not_set,About the job\n \n\n \nSalary: To be di...,7 days,0.0,NaN,2024-02-12 16:06:39.176582,NaN
3,Data Analyst,"['Lugano', 'Ticino', 'Switzerland']",Lugano,Switzerland,On-site,FORFIRM,not_set,not_set,About the job\n \n\n \nFORFIRM is provi...,14 days,0.0,NaN,2024-02-05 16:06:39.177600,NaN
4,Data Analyst - Logistics,"['Southampton', 'England', 'United Kingdom']",Southampton,United Kingdom,On-site,"Butler, Bridge & May",not_set,not_set,About the job\n \n\n \nLocation: Southa...,6 days,0.0,NaN,2024-02-13 16:06:39.177600,NaN


In [114]:
# Проверим результат:
df.groupby(['country','skills'])['applicants'].count().reset_index()

,country,skills,applicants
0,Amsterdam Area,data visualization,1
1,Brussels Metropolitan Area,data visualization,1
2,France,data visualization,3
3,France,python,1
4,France,sql,1
5,France,tableau,25
6,Germany,data visualization,6
7,Germany,power bi,2
8,Germany,python,2
9,Germany,sql,2


На этом этапе мы проанализировали описание ваканий и создали отдельные столбцы со скилами. Столбец принимает значение 0 или 1 в зависоимоти от наличия требований того или иного скила.

### Удалить ненужные атрибуты (признаки).

Для анализа нам не нужны: area, job_description, posted date, for_junior - удалим их
    

In [115]:
df=df.drop([ 'area', 'job_description', 'posted_date', 'for_junior' ], axis=1)

In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777 entries, 0 to 776
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   job_title     777 non-null    object        
 1   city          777 non-null    object        
 2   country       777 non-null    object        
 3   workplace     777 non-null    object        
 4   company_name  777 non-null    object        
 5   industry      777 non-null    object        
 6   company_size  777 non-null    object        
 7   applicants    777 non-null    float64       
 8   date          777 non-null    datetime64[ns]
 9   skills        109 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 60.8+ KB


Сохраним файл данных для дашборда

In [117]:
df.to_csv("~/Desktop/dash.csv")

## Дашборд

https://public.tableau.com/app/profile/alex2064/viz/Linkin/Dashboard1?publish=yes